In [ ]:
!apt install libgraphviz-dev
!pip install pygraphviz

In [ ]:
!pip install xlrd

In [ ]:
!pip install --upgrade pandas

In [ ]:
import networkx as nx
import graphviz
from graphviz import Graph, Source
from networkx.drawing.nx_agraph import to_agraph
from graphviz import Graph, Source

def gprint(g):


  B = np.array(g)

  G = nx.from_numpy_array(B, parallel_edges=True, create_using=nx.MultiGraph)

  G.graph['edge'] = {'arrowsize': '0.6', 'splines': 'curved'}
  G.graph['graph'] = {'scale': '3'}


  A = to_agraph(G)
  A.layout('dot')
  #A.draw('multi.png')
  Source(A)

In [ ]:
import numpy as np
import itertools as it

def dfs(g, visited, v=0):
    num_visited = 1
    visited[v] = True

    neighbors = []
    for idx, e in enumerate(g[v]):
      if e != 0 and idx != v:
        neighbors.append(idx)


    for u in neighbors:
        if not visited[u]:
            num_visited += dfs(g, visited, u)

    return num_visited

def ptr(n, k, l):
    if k < 1:
        return
    if k == 1:
        if n >= l:
            yield [n,]
        return
    for i in range(l, n + 1):
        for result in ptr(n-i, k-1, i):
            yield [i,] + result

def ptr_nu(n, k):

    if k < 1:
        return
    if k == 1:
        yield [n,]
        return
    for i in range(0, n + 1):
        for result in ptr_nu(n-i, k-1):
            yield [i,] + result

def sds(k):
    s = []
    d=1
    while 2*k + 2*d <= 6*k:
        s += [i for i in ptr(2*k+2*d, d, l=3)]
        d += 1
    return s

def rec(idx, g, sol, full_graphs):
    if g[idx-1][idx-1] % 2 != 0 :
        #print(idx, *g,  end='\n\n', sep='\n')
        return

    r = sol[idx] - sum(g[idx][:idx])

    if idx == len(sol)-1:
        if r >= 0:
            g[idx][idx] = r
            full_graphs.append(g.copy())
            return
        else:
            return
    for i in ptr_nu(r, len(sol) - idx):
        new_s = g[idx][:idx] + i
        g[idx] = new_s
        for s in enumerate(g[idx+1:]):
            g[s[0] + idx + 1] = s[1][0:idx] + [new_s[s[0] + idx + 1]] + s[1][idx+1:]
        rec(idx+1, g, sol, full_graphs)

def eq_str(l):
  return [index for index, value in enumerate(l) if value in set([i for i in l if l.count(i) > 1])]

def str_swap(m, per):
  new = []
  for i in per:
    new.append(m[i])
  return new

def swap(m, per):
  first = str_swap(m, per)
  tr_first = [list(i) for i in zip(*first)]
  second = str_swap(tr_first, per)
  new = [list(i) for i in zip(*second)]
  return new

def main(k):
    full_graphs = dict()

    for sol in sds(k)[:]:
        sol_graphs = []
        n = len(sol)

        if len(sol) == 1:
            full_graphs[tuple(sol)] = [[list(sol)]]
            continue
        rec(0, [[0]*n for i in range(n)], sol, sol_graphs)

        connected = []
        for g in sol_graphs:
            n = len(g)
            if n == dfs(g, [False]*n):
                connected.append(g)
            else:
                continue

        i = 0
        while i != len(connected):
          g = connected[i]
          for j in [k for k in it.permutations(range(n))][0:]:

            if j == tuple(range(n)):
              continue

            s = swap(g, j)

            if s == g:
              continue

            while s in connected:
              connected.pop(connected.index(s))

          i += 1

        full_graphs[tuple(sol)] = connected
        #print(sol, len(connected))

    return full_graphs

def mprint(m):
  for i in m:
    print(i, sep=', ')
  print()

def results(f):
  s = 0
  for i in f:

    print('степенная поледовательность: (2, ..., 2, '+','.join([str(j) for j in i])+')')
    print('количество неизоморфных стяжек:', len(f[i]))
    s += len(f[i])
    print()

  print()
  print('всего неизоморфных стяжек:', s)

def all_graphs(f):
  idx = 1
  for s in f.keys():
      for i in f[s]:
        print(idx, s)
        mprint(i)
        idx += 1

def all_graphs_sol(f, sol):
  idx = 1
  for s in sol:
    for i in f[s]:
      print(idx, s)
      mprint(i)
      idx += 1

def print_all_graphs(f, sol): #не работает
  idx = 1
  for s in sol:
    for i in f[s]:
      print(idx, s)
      gprint(i)
      idx += 1

In [ ]:
sds(4)

[[10],
 [3, 9],
 [4, 8],
 [5, 7],
 [6, 6],
 [3, 3, 8],
 [3, 4, 7],
 [3, 5, 6],
 [4, 4, 6],
 [4, 5, 5],
 [3, 3, 3, 7],
 [3, 3, 4, 6],
 [3, 3, 5, 5],
 [3, 4, 4, 5],
 [4, 4, 4, 4],
 [3, 3, 3, 3, 6],
 [3, 3, 3, 4, 5],
 [3, 3, 4, 4, 4],
 [3, 3, 3, 3, 3, 5],
 [3, 3, 3, 3, 4, 4],
 [3, 3, 3, 3, 3, 3, 4],
 [3, 3, 3, 3, 3, 3, 3, 3]]

In [ ]:
f = main(3)

In [ ]:
results(f)

степенная поледовательность: (2, ..., 2, 8)
количество неизоморфных стяжек: 1

степенная поледовательность: (2, ..., 2, 3,7)
количество неизоморфных стяжек: 2

степенная поледовательность: (2, ..., 2, 4,6)
количество неизоморфных стяжек: 2

степенная поледовательность: (2, ..., 2, 5,5)
количество неизоморфных стяжек: 3

степенная поледовательность: (2, ..., 2, 3,3,6)
количество неизоморфных стяжек: 6

степенная поледовательность: (2, ..., 2, 3,4,5)
количество неизоморфных стяжек: 10

степенная поледовательность: (2, ..., 2, 4,4,4)
количество неизоморфных стяжек: 4

степенная поледовательность: (2, ..., 2, 3,3,3,5)
количество неизоморфных стяжек: 14

степенная поледовательность: (2, ..., 2, 3,3,4,4)
количество неизоморфных стяжек: 22

степенная поледовательность: (2, ..., 2, 3,3,3,3,4)
количество неизоморфных стяжек: 30

степенная поледовательность: (2, ..., 2, 3,3,3,3,3,3)
количество неизоморфных стяжек: 17


всего неизоморфных стяжек: 111


In [ ]:
all_graphs(f)

### Удаление лишних стяжек



In [ ]:
def tr_main(g):
  res = False
  for idx, s in enumerate(g):
    if sum(s) > 3:
      if s[idx] > 0:
        res = True
      elif max(s) > 1:
        res = True
  return res

In [ ]:
f_new = dict()

for i in f:
  new = []
  for g in f[i]:
    if tr_main(g):
      continue
    else:
      new.append(g)

  f_new[i] = new

In [ ]:
results(f_new)

степенная поледовательность: (2, ..., 2, 8)
количество неизоморфных стяжек: 0

степенная поледовательность: (2, ..., 2, 3,7)
количество неизоморфных стяжек: 0

степенная поледовательность: (2, ..., 2, 4,6)
количество неизоморфных стяжек: 0

степенная поледовательность: (2, ..., 2, 5,5)
количество неизоморфных стяжек: 0

степенная поледовательность: (2, ..., 2, 3,3,6)
количество неизоморфных стяжек: 0

степенная поледовательность: (2, ..., 2, 3,4,5)
количество неизоморфных стяжек: 0

степенная поледовательность: (2, ..., 2, 4,4,4)
количество неизоморфных стяжек: 0

степенная поледовательность: (2, ..., 2, 3,3,3,5)
количество неизоморфных стяжек: 0

степенная поледовательность: (2, ..., 2, 3,3,4,4)
количество неизоморфных стяжек: 0

степенная поледовательность: (2, ..., 2, 3,3,3,3,4)
количество неизоморфных стяжек: 5

степенная поледовательность: (2, ..., 2, 3,3,3,3,3,3)
количество неизоморфных стяжек: 17


всего неизоморфных стяжек: 22


In [ ]:
all_graphs_sol(f_new, [(3, 3, 3, 3, 4)])

1 (3, 3, 3, 3, 4)
[0, 0, 0, 2, 1]
[0, 0, 2, 0, 1]
[0, 2, 0, 0, 1]
[2, 0, 0, 0, 1]
[1, 1, 1, 1, 0]

2 (3, 3, 3, 3, 4)
[0, 0, 0, 2, 1]
[0, 2, 0, 0, 1]
[0, 0, 2, 0, 1]
[2, 0, 0, 0, 1]
[1, 1, 1, 1, 0]

3 (3, 3, 3, 3, 4)
[0, 0, 1, 1, 1]
[0, 0, 1, 1, 1]
[1, 1, 0, 0, 1]
[1, 1, 0, 0, 1]
[1, 1, 1, 1, 0]

4 (3, 3, 3, 3, 4)
[0, 0, 1, 1, 1]
[0, 2, 0, 0, 1]
[1, 0, 0, 1, 1]
[1, 0, 1, 0, 1]
[1, 1, 1, 1, 0]

5 (3, 3, 3, 3, 4)
[2, 0, 0, 0, 1]
[0, 2, 0, 0, 1]
[0, 0, 2, 0, 1]
[0, 0, 0, 2, 1]
[1, 1, 1, 1, 0]



### Рисование


In [ ]:
num = 0
for i in f:
  for j in f[i]:
    num += 1
    g = j

    for n, i  in enumerate(g):
      i[n] = int(i[n]/2)
      g[n] = i

    B = np.array(g)

    G = nx.from_numpy_array(B, parallel_edges=True, create_using=nx.MultiGraph)

    G.graph['edge'] = {'arrowsize': '0.6', 'splines': 'curved'}
    G.graph['graph'] = {'scale': '3'}


    A = to_agraph(G)
    A.layout('dot')
    A.draw(f'/content/drive/MyDrive/all_graphs/multi_{num}.png')
    #Source(A)